<a href="https://colab.research.google.com/github/ods199812-afk/Bayesian-/blob/main/Mental_Math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import math
import string
# =========================
# 工具函数
# =========================


LETTER2NUM = {ch: i+1 for i, ch in enumerate(string.ascii_uppercase)}
NUM2LETTER = {v: k for k, v in LETTER2NUM.items()}

def gen_letter_calc_question():
    op = random.choice(["+", "-", "×", "÷"])

    if op in ["+", "-"]:
        a = random.choice(string.ascii_uppercase)
        b = random.choice(string.ascii_uppercase)
        av, bv = LETTER2NUM[a], LETTER2NUM[b]
        ans = av + bv if op == "+" else av - bv
        q = f"{a} {op} {b} = ?"
        hint = f"映射：{a}={av}, {b}={bv}"
        return q, [str(ans)], hint

    if op == "×":
        a = random.choice("ABCDEFGHIJ")
        b = random.choice("ABCDEFGHIJ")
        av, bv = LETTER2NUM[a], LETTER2NUM[b]
        ans = av * bv
        q = f"{a} × {b} = ?"
        hint = f"映射：{a}={av}, {b}={bv}"
        return q, [str(ans)], hint

    # ÷：保证整除
    b = random.choice("ABCDEFGH")
    bv = LETTER2NUM[b]
    ans = random.randint(2, 10)
    av = bv * ans
    while av > 26:
        ans = random.randint(2, 10)
        av = bv * ans
    a = NUM2LETTER[av]
    q = f"{a} ÷ {b} = ?"
    hint = f"映射：{a}={av}, {b}={bv}"
    return q, [str(ans)], hint



# =========================
# 题目生成器
# =========================

def gen_arithmetic_question():
    """
    随机生成一题加减乘除，返回 (题目字符串, 正确答案token列表)
    """
    op = random.choice(['+', '-', '*', '÷'])

    if op in ['+', '-']:
        a = random.randint(10, 999)
        b = random.randint(10, 999)
        if op == '-':
            # 保证结果非负，减法更适合心算
            if b > a:
                a, b = b, a
        if op == '+':
            ans = a + b
        else:
            ans = a - b
        question = f"{a} {op} {b} = ?"

    elif op == '*':
        a = random.randint(10, 99)
        b = random.randint(10, 99)
        ans = a * b
        question = f"{a} × {b} = ?"

    else:  # 除法，保证整除
        b = random.randint(2, 20)
        ans = random.randint(2, 50)
        a = b * ans
        question = f"{a} ÷ {b} = ?"

    return question, [str(ans)]

def gen_exponent_question():
    """
    生成类似 2^x = 32768 求 x 的题
    """
    base = random.choice([2, 3, 4, 5, 6, 7, 8, 9, 10])
    x = random.randint(2, 10)
    value = base ** x
    question = f"{base}^x = {value}，求 x："
    return question, [str(x)]

def gen_percent_mult_question():
    """
    生成 x * r 形式的心算题，r 选一些常见好算的比例
    返回：题目, [答案字符串]
    """
    r = random.choice([0.75, 0.25, 0.5, 1.5, 1.25, 0.2, 0.15, 0.12, 0.08, 0.06, 0.04])
    x = random.choice([random.randint(20, 400), random.randint(500, 3000)])

    # 让答案尽量整齐：必要时把 x 调成 r 的友好倍数
    # 例如 0.75 = 3/4 => x 取 4 的倍数更好算
    if r in [0.75, 0.25]:
        x = (x // 4) * 4
    elif r in [0.15]:
        x = (x // 20) * 20  # 0.15 = 15% => 20的倍数更常整齐
    elif r in [0.12]:
        x = (x // 25) * 25
    elif r in [0.08, 0.04]:
        x = (x // 25) * 25

    ans = x * r

    # 如果是整数就不要小数点
    if abs(ans - round(ans)) < 1e-9:
        ans_str = str(int(round(ans)))
    else:
        # 保留最多2位，去掉末尾0
        ans_str = f"{ans:.2f}".rstrip("0").rstrip(".")

    q = f"{x} × {r} = ?"
    return q, [ans_str]


def gen_fib_random_skip_question():
    """
    随机截取 Fibonacci 的一段
    随机隐藏后 k 项
    只问第 k 项（不是全写）
    """
    # 先生成一长一点的 fib
    fib = [0, 1]
    while len(fib) < 20:
        fib.append(fib[-1] + fib[-2])

    # 随机选一个起点（不要太靠后）
    start = random.randint(2, 8)
    shown_len = random.randint(6, 8)   # 给你看的长度
    k = random.randint(3, 6)            # 问第几项

    shown = fib[start:start + shown_len]

    # 算答案：从最后两个开始推 k 步
    a, b = shown[-2], shown[-1]
    for _ in range(k):
        a, b = b, a + b
    answer = b

    q = (
        ", ".join(map(str, shown))
        + ", " + ", ".join(["x"] * (k - 1))
        + ", ?\n"
        + f"只写出最后这个 ?（即接下来的第 {k} 项）："
    )

    return q, [str(answer)]



def gen_exp_approx_question():
    """
    训练 e^x 的近似，要求保留2位小数
    """
    # 常见小范围，比较适合心算近似
    x = random.choice([-2, -1.5, -1, -0.5, -0.2, 0.2, 0.5, 1, 1.5, 2, 2.5, 3])
    val = math.exp(x)
    ans = f"{val:.2f}"  # 2 位小数
    q = f"近似计算：e^{x} ≈ ? （保留2位小数）"
    return q, [ans]


def run_exponent_drill(n=30):
    correct = 0
    done = 0
    t0 = time.time()

    for i in range(1, n + 1):
        q, ans = gen_exponent_question()   # ✅ 只解包 2 个
        print(f"\n第{i}题：{q}")

        user = input("你的答案：").strip()
        if user.lower() == "q":
            break

        done += 1
        if user == ans[0]:
            print("✅ 正确")
            correct += 1
        else:
            print(f"❌ 错误，正确：{ans[0]}")

    t1 = time.time()
    if done == 0:
        print("\n本轮没有完成任何题。")
        return

    print(f"\n完成 {done} 题，正确 {correct}，正确率 {correct/done:.1%}，用时 {t1-t0:.1f}s")

def run_letter_drill(n=30):
    correct = 0
    done = 0
    print(f"=== 字母→数字计算 | {n} 题（输入 q 退出）===\n")

    for i in range(1, n+1):
        q, ans_tokens, hint = gen_letter_calc_question()  # ✅ 三个
        print(f"第{i}题：{q}   (A=1,...,Z=26)")
        user = input("你的答案：").strip()
        if user.lower() == "q":
            break

        done += 1
        print(hint)  # ✅ 显示映射

        if user == ans_tokens[0]:
            print("✅ 正确\n")
            correct += 1
        else:
            print(f"❌ 错误，正确：{ans_tokens[0]}\n")

    if done:
        print(f"总结：{correct}/{done} = {correct/done:.1%}")



def run_drill(mode="percent", n=30):
    """
    mode:
      - "percent": x * 0.75 等
      - "fib5": Fibonacci 后5项
      - "exp": e^x 近似
    """
    gen_map = {
        "percent": gen_percent_mult_question,
        "fib5": gen_fib_random_skip_question,
        "exp": gen_exp_approx_question,
        "letter" : gen_letter_calc_question,
    }
    gen = gen_map[mode]

    correct = 0
    done = 0
    t0 = time.time()

    print(f"=== 专项训练：{mode} | 共 {n} 题（输入 q 退出）===\n")

    for i in range(1, n + 1):
        q, ans_tokens = gen()
        print(f"第{i}题：\n{q}")
        user = input("你的答案：").strip()
        if user.lower() == "q":
            break

        done += 1
        user_tokens = user.replace(",", " ").split()

        ok = (len(user_tokens) == len(ans_tokens))
        if ok:
            for ut, at in zip(user_tokens, ans_tokens):
                # exp题我们允许误差：±0.03（因为你是心算近似）
                if mode == "exp":
                    try:
                        ok = ok and (abs(float(ut) - float(at)) <= 0.03)
                    except:
                        ok = False
                else:
                    ok = ok and (ut == at)

        if ok:
            print("✅ 正确\n")
            correct += 1
        else:
            print("❌ 错误。正确答案：", " ".join(ans_tokens), "\n")

    t1 = time.time()
    if done == 0:
        print("本轮没有完成任何题。")
        return
    print(f"=== 总结：完成 {done} 题，正确 {correct}，正确率 {correct/done:.1%}，用时 {t1-t0:.1f}s ===")


In [ ]:
run_drill("fib5")

=== 专项训练：fib5 | 共 30 题（输入 q 退出）===

第1题：
2, 3, 5, 8, 13, 21, 34, x, x, ?
只写出最后这个 ?（即接下来的第 3 项）：
你的答案：144
✅ 正确

第2题：
13, 21, 34, 55, 89, 144, 233, 377, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 5 项）：
你的答案：4181
✅ 正确

第3题：
21, 34, 55, 89, 144, 233, 377, 610, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 5 项）：
你的答案：7765
❌ 错误。正确答案： 6765 

第4题：
13, 21, 34, 55, 89, 144, x, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 6 项）：
你的答案：2584
✅ 正确

第5题：
8, 13, 21, 34, 55, 89, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 5 项）：
你的答案：987
✅ 正确

第6题：
1, 2, 3, 5, 8, 13, 21, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 5 项）：
你的答案：144
❌ 错误。正确答案： 233 

第7题：
21, 34, 55, 89, 144, 233, 377, 610, x, x, x, ?
只写出最后这个 ?（即接下来的第 4 项）：
你的答案：4181
✅ 正确

第8题：
8, 13, 21, 34, 55, 89, 144, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 5 项）：
你的答案：1584
❌ 错误。正确答案： 1597 

第9题：
2, 3, 5, 8, 13, 21, x, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 6 项）：
你的答案：377
✅ 正确

第10题：
2, 3, 5, 8, 13, 21, x, x, x, ?
只写出最后这个 ?（即接下来的第 4 项）：
你的答案：144
✅ 正确

第11题：
1, 2, 3, 5, 8, 13, 21, x, x, x, x, ?
只写出最后这个 ?（即接下来的第 5 项）：
你的答案：233
✅ 正确

第12题：
5,